In [ ]:
import os
import pandas as pd 
import numpy as np 
import sys
import glob
from Bio import SeqIO
import re

In [ ]:
os.chdir('/fs/ess/PAS0439/MING/virome/amg_analysis/dram_annotation')

In [ ]:
amg = pd.read_csv('amg_contig_individual_check.tsv', sep = '\t')

In [ ]:
amg.contig_id = amg.contig_id.apply(lambda x: x.replace('__', '||'))

In [ ]:
filtering_contigs = list(np.loadtxt('aj_filtered_contigs.txt', dtype='str'))

In [ ]:
amg_filtering = amg[amg.contig_id.isin(filtering_contigs)]

In [ ]:
amg_filtering.fillna('', inplace=True)

In [ ]:
amg_filtering.loc[:, 'hits'] = amg_filtering['kegg_hit'] + amg_filtering['cazy_hits']  + amg_filtering['viral_hit']  + amg_filtering['pfam_hits'] + amg_filtering['vogdb_hit']

In [ ]:
#pd.set_option('display.max_rows', 700)
#gh_df = amg_filtering[amg_filtering.hits.str.contains('GH\d', regex=True)]
#gh_df_filter_virus = gh_df[~gh_df.amg_flags.str.contains('A|P')]
#gh_df_filter_virus_CBM_GH10_16 = gh_df_filter_virus[~gh_df_filter_virus.hits.str.contains('CBM|GH16|GH10 ')].query('contig_length < 200000')
#gh_containing= list(set(gh_df_filter_virus_CBM_GH10_16['contig_id']))

In [ ]:
LPS = ['carbohydrate kinase', 'carbohydrate-kinase', 'glycosyltransferase family 11 ', 'glycosyltransferase family 25', 'glycosyl transferase family 11 ', 'glycosyl transferase family 25', 'Glycosyltransferase family 11 ', 'Glycosyltransferase family 25', 'GT11 ', 'GT25', 'nucleotide sugar epimerase', 'nucleotide sugar-epimerase', 'nucleotide-sugar epimerase', 'nucleotide-sugar-epimerase', 'nucleotidyltransferase', 'nucleotidyl transferase', 'nucleotidyl-transferase', 'plasmid stability', 'endonuclease']

In [ ]:
def filter_lps():
    furthur_check = []
    for index, row in amg_filtering.iterrows():
        hits = row['hits']
        votu = row['contig_id']
        if votu in furthur_check:
            continue
        else:
            if hits == '':
                continue
            else:
                for lps in LPS:

                    if lps in hits:
                        furthur_check.append(votu)
                    else:
                        continue
    
    return furthur_check
                    

In [ ]:
furthur_check = filter_lps()

In [ ]:
with open('/fs/ess/PAS0439/MING/virome/checkv_trimmed_for_dowmstream/AMG/supplemental_material/LPS_containing.txt', 'w') as outfile:
    for f in furthur_check:
        outfile.write(f + '\n')

In [ ]:
amg_check = amg_filtering[amg_filtering.contig_id.isin(furthur_check)].sort_values('contig_length')

In [ ]:
amg_clean = amg_filtering[~amg_filtering.contig_id.isin(furthur_check)].sort_values('contig_length').reset_index(drop=True)

In [ ]:
amg_clean.to_csv('/fs/ess/PAS0439/MING/virome/checkv_trimmed_for_dowmstream/AMG/supplemental_material/clean_amg_contigs_full_annotation.tsv', sep ='\t', index=None)

In [ ]:
pd.set_option('display.max_rows', 900)

In [ ]:
amg_clean_contigs = list(set(amg_clean['contig_id']))

In [ ]:
with open('/fs/ess/PAS0439/MING/virome/checkv_trimmed_for_dowmstream/AMG/clean_amg_contigs.txt', 'w') as outfile:
    for f in amg_clean_contigs:
        outfile.write(f + '\n')

In [ ]:
pd.set_option('display.max_rows', 700)
gh_df = amg_clean[amg_clean.hits.str.contains('GH\d', regex=True)]
gh_df_filter_virus = gh_df[~gh_df.amg_flags.str.contains('A|P')]
gh_df_filter_virus_CBM = gh_df_filter_virus[~gh_df_filter_virus.hits.str.contains('CBM')]
gh_df_filter_virus_CBM
#gh_containing= list(set(gh_df_filter_virus_CBM_GH10_16['contig_id']))